# New Section

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from fuzzywuzzy import fuzz

In [ ]:
peminjaman = pd.read_csv("/content/trx.csv")
member = pd.read_csv("/content/member prodi.csv")
koleksi = pd.read_csv("/content/koleksi yang dipinjam.csv")


df = member.merge(peminjaman, on="member_id", how="left")
df = df.merge(koleksi, on="Collection_id", how="left")



In [ ]:
df["combined"] = df["Title"].fillna("") + " " + df["Subject"].fillna("")


In [ ]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df["combined"])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
idx_map = {t: i for i, t in enumerate(df["Title"])}

In [ ]:
def rekomendasi_buku_member(member_id, top_n=10, w_tfidf=0.3, w_prodi=0.5, w_cf=0.2):
    if member_id not in df["member_id"].values:
        return pd.DataFrame(), "❌ member_id tidak ditemukan"

    data_member = df[df["member_id"] == member_id].iloc[0]
    prodi_id = data_member["ProgramStudi_id"]
    nama_member = data_member["Nama"]

    # Buku yang sudah pernah dipinjam
    buku_user = df[df["member_id"] == member_id]["Title"].dropna().unique().tolist()

    # --- 1. TF-IDF berdasarkan buku yang pernah dipinjam ---
    df_titles = df[["Title", "Subject"]].drop_duplicates().reset_index(drop=True)
    df_titles["score_tfidf"] = 0.0

    if len(buku_user) > 0:
        idx_user_books = [idx_map[b] for b in buku_user if b in idx_map]
        if len(idx_user_books) > 0:
            sim_scores = cosine_sim[idx_user_books].mean(axis=0)

            # Pastikan panjang sim_scores cocok dengan jumlah judul unik
            df_titles = df_titles.copy()
            df_titles["score_tfidf"] = 0.0  # inisialisasi kolom skor
            for i, title in enumerate(df_titles["Title"].unique()):
                if i < len(sim_scores):
                    df_titles.loc[df_titles["Title"] == title, "score_tfidf"] = sim_scores[i]


    # --- 2. Skor berdasarkan Prodi (frekuensi pinjaman di prodi tsb) ---
    df_prodi = df[df["ProgramStudi_id"] == prodi_id]
    popular_prodi = (
    df_prodi["Title"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "Title", "Title": "JumlahDipinjam"})
    )
    popular_prodi.columns = ["Title", "JumlahDipinjam"]  # pastikan kolomnya konsisten
    df_titles = df_titles.merge(popular_prodi, on="Title", how="left").fillna({"JumlahDipinjam": 0})
    df_titles["score_prodi"] = df_titles["JumlahDipinjam"] / df_titles["JumlahDipinjam"].max()

    # --- 3. Skor Collaborative Filtering sederhana ---
    user_activity = df["member_id"].value_counts()
    user_weights = df["member_id"].map(user_activity)
    df["weight"] = user_weights / user_weights.max()
    df_cf = df.groupby("Title")["weight"].mean().reset_index()
    df_cf.rename(columns={"weight": "score_cf"}, inplace=True)
    df_titles = df_titles.merge(df_cf, on="Title", how="left").fillna({"score_cf": 0})

    # --- 4. Gabungkan semua skor ---
    df_titles["final_score"] = (
        w_tfidf * df_titles["score_tfidf"]
        + w_prodi * df_titles["score_prodi"]
        + w_cf * df_titles["score_cf"]
    )

    # Hapus buku yang sudah pernah dipinjam user
    hasil = df_titles[~df_titles["Title"].isin(buku_user)]
    hasil = hasil.sort_values("final_score", ascending=False).head(top_n)

    print(f"\n📚 Personalized Recommendation untuk member_id={member_id} ({nama_member}):")
    print("   Berdasarkan kombinasi perilaku, isi buku, dan prodi (CF + TF-IDF + Prodi)\n")
    return hasil, "✅ OK"

#testtt


In [ ]:
# hasil = rekomendasi_buku_member(229375, top_n=10)
# print(hasil)


In [ ]:
sample_members = df["member_id"].drop_duplicates().sample(n=50, random_state=42)


In [ ]:
def is_match(title1, title2, threshold=85):
    """Cocokkan 2 judul buku secara fuzzy (toleransi ejaan berbeda)"""
    return fuzz.ratio(title1.lower(), title2.lower()) >= threshold

In [ ]:
def evaluasi_rekomendasi(df_all, user_subset, top_n=10, threshold=80):
    """
    Evaluasi sistem rekomendasi dengan Fuzzy Matching.
    threshold = ambang kemiripan judul (0–100)
    """
    hasil_eval = []
    n_user = 0
    print(f"🔎 Evaluasi {len(user_subset)} member...\n")

    for uid in user_subset:
        # Buku yang benar-benar pernah dipinjam (ground truth)
        buku_aktual = df_all[df_all["member_id"] == uid]["Title"].dropna().unique()
        if len(buku_aktual) == 0:
            continue

        # Hasil rekomendasi
        hasil, _ = rekomendasi_buku_member(uid, top_n=top_n)
        if hasil is None or hasil.empty:
            continue

        buku_pred = hasil["Title"].dropna().unique()
        if len(buku_pred) == 0:
            continue

        # Hitung kemiripan judul dengan fuzzy match
        matched = 0
        for t_pred in buku_pred:
            for t_actual in buku_aktual:
                if fuzz.partial_ratio(str(t_pred).lower(), str(t_actual).lower()) >= threshold:
                    matched += 1
                    break  # satu match sudah cukup

        precision = matched / len(buku_pred)
        recall = matched / len(buku_aktual)
        hasil_eval.append({"member_id": uid, "precision": precision, "recall": recall})
        n_user += 1

        print(f"👤 Member {uid}: Precision={precision:.2f}, Recall={recall:.2f}, Buku Aktual={len(buku_aktual)}")

    if n_user == 0:
        print("⚠️ Tidak ada member yang valid untuk evaluasi.")
        return pd.DataFrame()

    df_eval = pd.DataFrame(hasil_eval)
    print("\n📈 Hasil Evaluasi Keseluruhan:")
    print(f"Rata-rata Precision: {df_eval['precision'].mean():.3f}")
    print(f"Rata-rata Recall: {df_eval['recall'].mean():.3f}")

    return df_eval

In [ ]:
import random
df_prodi = df[["ProgramStudi_id"]].drop_duplicates().reset_index(drop=True)
df_prodi["NamaProdi"] = "Prodi " + df_prodi["ProgramStudi_id"].astype(str)
def evaluasi_rekomendasi_detail(df_all, user_subset, top_n=10, sample_size=5):
    hasil_evaluasi = []
    contoh_hasil = []

    for uid in user_subset:
        # --- Ambil data user & prodi ---
        data_user = df_all[df_all["member_id"] == uid][["ProgramStudi_id", "Nama"]].dropna().head(1)
        if data_user.empty:
            continue
        user_prodi_id = str(data_user["ProgramStudi_id"].values[0])
        user_nama = data_user["Nama"].values[0]
        user_prodi_nama = df_prodi.loc[
            df_prodi["ProgramStudi_id"].astype(str) == user_prodi_id, "NamaProdi"
        ].values
        user_prodi_nama = user_prodi_nama[0] if len(user_prodi_nama) > 0 else "Tidak diketahui"

        # --- Buku aktual (yang pernah dipinjam user ini) ---
        buku_aktual = df_all[df_all["member_id"] == uid]["Title"].dropna().unique()
        if len(buku_aktual) == 0:
            continue

        # --- Buku hasil rekomendasi ---
        hasil, status = rekomendasi_buku_member(uid, top_n=top_n)
        if hasil.empty:
            continue

        # --- Ambil prodi buku rekomendasi dan nama prodinya ---
        hasil = hasil.merge(
            df_all[["Title", "ProgramStudi_id"]].drop_duplicates(),
            on="Title",
            how="left"
        ).merge(
            df_prodi,
            on="ProgramStudi_id",
            how="left"
        )

        buku_rekom = hasil["Title"].values
        relevan = set(buku_rekom).intersection(set(buku_aktual))
        precision = len(relevan) / len(buku_rekom) if len(buku_rekom) > 0 else 0
        recall = len(relevan) / len(buku_aktual) if len(buku_aktual) > 0 else 0

        # --- Hitung proporsi buku dari prodi yang sama ---
        same_prodi = hasil[hasil["ProgramStudi_id"].astype(str) == str(user_prodi_id)]
        persen_prodi_sama = len(same_prodi) / len(hasil) * 100 if len(hasil) > 0 else 0

        hasil_evaluasi.append({
            "member_id": uid,
            "nama": user_nama,
            "prodi_user": user_prodi_nama,
            "precision": precision,
            "recall": recall,
            "persentase_prodi_sama": round(persen_prodi_sama, 2)
        })

        contoh_hasil.append({
            "member_id": uid,
            "nama": user_nama,
            "prodi_user": user_prodi_nama,
            "buku_aktual": list(buku_aktual[:10]),
            "buku_rekomendasi": list(buku_rekom[:10]),
            "prodi_buku": list(hasil["NamaProdi"].fillna("Tidak diketahui").values[:10]),
            "precision": round(precision, 3),
            "recall": round(recall, 3),
            "persen_prodi_sama": round(persen_prodi_sama, 2)
        })

    df_eval = pd.DataFrame(hasil_evaluasi)

    # --- Ringkasan ---
    print("📈 Hasil Evaluasi Keseluruhan:")
    print("Rata-rata Precision:", round(df_eval["precision"].mean(), 4))
    print("Rata-rata Recall:", round(df_eval["recall"].mean(), 4))
    print("Rata-rata % Buku dari Prodi Sama:", round(df_eval["persentase_prodi_sama"].mean(), 2), "%")

    # --- Contoh detail beberapa user ---
    print("\n🔍 Contoh Hasil Evaluasi Detail ({} member):".format(sample_size))
    sample_users = random.sample(contoh_hasil, min(sample_size, len(contoh_hasil)))

    for item in sample_users:
        print(f"\n👤 Member ID: {item['member_id']} - {item['nama']} ({item['prodi_user']})")
        print(f"🎯 Precision: {item['precision']}, Recall: {item['recall']}")
        print(f"🏫 % Rekomendasi dari Prodi Sama: {item['persen_prodi_sama']}%")
        print("📚 Buku Aktual:")
        for b in item['buku_aktual']:
            print("   •", b)
        print("🤖 Rekomendasi Sistem (dengan nama prodi):")
        for b, p in zip(item['buku_rekomendasi'], item['prodi_buku']):
            print(f"   • {b}  ({p})")

    return df_eval

In [ ]:
import numpy as np
user_ids = df["member_id"].dropna().unique()
user_ids = np.random.choice(user_ids, size=5, replace=False)

# Jalankan evaluasi
hasil_eval_detail = evaluasi_rekomendasi_detail(df, user_ids, top_n=10, sample_size=5)
# hasil_eval = evaluasi_rekomendasi(df, user_ids, top_n=100)


📚 Personalized Recommendation untuk member_id=169800 (S1 FARMASI):
   Berdasarkan kombinasi perilaku, isi buku, dan prodi (CF + TF-IDF + Prodi)


📚 Personalized Recommendation untuk member_id=171173 (S1 Ilmu Komunikasi):
   Berdasarkan kombinasi perilaku, isi buku, dan prodi (CF + TF-IDF + Prodi)


📚 Personalized Recommendation untuk member_id=226562 (S1 Manajemen):
   Berdasarkan kombinasi perilaku, isi buku, dan prodi (CF + TF-IDF + Prodi)


📚 Personalized Recommendation untuk member_id=170589 (S1 Sosiologi):
   Berdasarkan kombinasi perilaku, isi buku, dan prodi (CF + TF-IDF + Prodi)


📚 Personalized Recommendation untuk member_id=204650 (S2 Kenotariatan):
   Berdasarkan kombinasi perilaku, isi buku, dan prodi (CF + TF-IDF + Prodi)

📈 Hasil Evaluasi Keseluruhan:
Rata-rata Precision: 0.0
Rata-rata Recall: 0.0
Rata-rata % Buku dari Prodi Sama: 53.33 %

🔍 Contoh Hasil Evaluasi Detail (5 member):

👤 Member ID: 170589 - S1 Sosiologi (Prodi 17)
🎯 Precision: 0.0, Recall: 0.0
🏫 % Rekomenda